In [1]:
import sys; print('Python %s on %s' % (sys.version, sys.platform))
sys.path.extend(['./cnn_finetune'])

from load_cifar import load_cifar10_data, load_cifar100_data

import numpy as np; print('numpy:', np.__version__)
import sklearn; print('sklearn:', sklearn.__version__)
import tensorflow as tf; print('tensorflow', tf.__version__)
import keras; print('keras', keras.__version__)

Python 3.5.2 (default, Nov 12 2018, 13:43:14) 
[GCC 5.4.0 20160609] on linux


Using TensorFlow backend.


numpy: 1.16.2
sklearn: 0.20.3
tensorflow 1.13.1
keras 2.2.4


In [2]:
import os
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
from keras.utils import multi_gpu_model
# set gpu visible environment variable
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

config = tf.ConfigProto()

# cpu
# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores, inter_op_parallelism_threads=num_cores, allow_soft_placement=True, device_count={'CPU': 4})
    
# gpu allow_growth
config.gpu_options.allow_growth = True

# set memory limit of gpu
# config.gpu_options.per_process_gpu_memory_fraction = 0.5

set_session(tf.Session(config=config))

In [3]:
from keras.optimizers import SGD, Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping, ReduceLROnPlateau, TensorBoard, CSVLogger

from cnn_finetune.densenet1612019 import densenet161_model
from lib.KerasImageGenerator import *

In [12]:
# Example to fine-tune on 3000 samples from Cifar10
model_name = 'densenet_161'
dataset_name = 'imaterialist'
img_rows, img_cols = 224, 224 # Resolution of inputs
channel = 3
num_classes = 2019
batch_size = 16
epochs = 2000

load_weights = False
fine_tuning = False

In [13]:
Val_Image_Keras_Generator = KerasImageGenerator(ds='val',corp = False,batch_size = batch_size)
Train_Image_Keras_Generator = KerasImageGenerator(ds='train',corp = False,batch_size = batch_size)


Found 10095 images belonging to 2019 classes.
Found 1008641 images belonging to 2019 classes.


In [14]:
Val_image_generator_Iterator = Val_Image_Keras_Generator.keras_get_images_iter()
Train_image_generator_Iterator = Train_Image_Keras_Generator.keras_get_images_iter()

In [15]:
steps_per_epoch = len(Train_image_generator_Iterator)
validation_steps = len(Val_image_generator_Iterator)

In [16]:
print(steps_per_epoch)
print(validation_steps)

63041
631


In [17]:


# Load our model
model = densenet161_model(img_rows, img_cols, channel, 
                          num_classes=num_classes, fine_tuning=fine_tuning, load_weights=load_weights)

dense161_model: classes: 2019 fine-tuning: False
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding2 (None, 230, 230, 3)  0           data[0][0]                       
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 96) 14112       conv1_zeropadding[0][0]          
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 96) 384         conv1[0][0]                      
____________________________________________________________

In [ ]:
# Learning rate is changed to 0.001
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8, decay=0.0, amsgrad=False)
sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
# parallel_model = multi_gpu_model(model, gpus=2)
# parallel_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['acc'])
checkpoint = ModelCheckpoint("./models/" + model_name + "/"+ dataset_name +"-e" + "{epoch:02d}-{val_loss:.4f}.hd5", 
                             monitor='val_loss', 
                             verbose=1, 
                             save_best_only=True, 
                             save_weights_only=True, 
                             mode='auto', 
                             period=1)

checkpoint_best = ModelCheckpoint("./models/" + model_name + "/"+ dataset_name + "-best.hd5", 
                                  monitor='val_loss', 
                                  verbose=1, 
                                  save_best_only=True, 
                                  save_weights_only=True,
                                  mode='auto', 
                                  period=1)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              min_delta=0, 
                              patience=20, 
                              verbose=1, 
                              mode='auto', 
                              baseline=None, 
                              restore_best_weights=False)

reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, verbose=1, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

tensorBoard = TensorBoard(log_dir='./models/' + model_name +'/logs', histogram_freq=0, write_graph=True, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

csvLogger = CSVLogger(filename='./models/'+ model_name + '/' + dataset_name + "-log.csv", separator=',', append=False)

# Start Fine-tuning
model.fit_generator(Train_image_generator_Iterator,
          steps_per_epoch = steps_per_epoch,
          epochs=epochs,
          shuffle=True,
          verbose=1,
          validation_data=Val_image_generator_Iterator,
          validation_steps=validation_steps,
          callbacks=[checkpoint, checkpoint_best, reduceLR, earlyStopping, tensorBoard, csvLogger]
          )

Epoch 1/2000
18504/63041 [=======>......................] - ETA: 19:31:10 - loss: 7.3796 - acc: 0.0040

In [ ]:
# Make predictions
#predictions_valid = model.predict(X_valid, batch_size=batch_size, verbose=1)
#print(Y_valid.shape, predictions_valid.shape)

In [ ]:
# import numpy as np
# np.set_printoptions(threshold=np.inf)
# print(Y_valid)
# print(predictions_valid)

In [ ]:
#print('cross-entropy:', sklearn.metrics.log_loss(Y_valid, predictions_valid))

#y_true = np.argmax(Y_valid, axis=1)
#y_hat = np.argmax(predictions_valid, axis=1)
# print('report:', sklearn.metrics.classification_report(y_true, y_hat))
#print('acc:', sklearn.metrics.accuracy_score(y_true, y_hat))
# print('roc:', sklearn.metrics.roc_auc_score(y_true, y_hat))
# print('confusion_matrix:\n', sklearn.metrics.confusion_matrix(y_true, y_hat))